In [1]:
%pip install duckdb pandas matplotlib numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 2.4 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached pandas-2.3.3-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (12.8 MB)
  Using cached matplotlib-3.10.7-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (8.7 MB)
  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached pillow-12.0.0-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (7.0 MB)
  Using cached contourpy-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (325 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl (113 kB)
  Using cached fonttools-4.60.1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (4.8 MB)
  Using cached kiwisolver-1.4.9-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_

In [2]:
import duckdb
import pandas as pd

con = duckdb.connect('mobility.duckdb')


def SQL(q):
    """Run SQL (printed for clarity) and return a DataFrame."""
    # print("\nSQL>\n" + textwrap.dedent(q).strip())
    return con.execute(q).fetchdf()


print("DuckDB version:", con.sql("SELECT version();").fetchone()[0])

DuckDB version: v1.4.2


In [11]:
SQL("""
    SELECT count(*)
    FROM '20230701_Viajes_distritos.csv'
    GROUP BY ALL
""")

,count_star()
0,18559829


In [33]:
SQL("""
    SELECT
        * EXCLUDE (estudio_destino_posible, estudio_origen_posible),
        CASE estudio_destino_posible WHEN 'si' THEN TRUE ELSE FALSE END AS estudio_destino_posible,
        CASE estudio_origen_posible  WHEN 'si' THEN TRUE ELSE FALSE END AS estudio_origen_posible
    FROM read_csv_auto(
        '20230701_Viajes_distritos.csv',
        types = {
            'estudio_destino_posible' : 'VARCHAR',
            'estudio_origen_posible'  : 'VARCHAR'
        }
    )
    USING SAMPLE 0.001 % (BERNOULLI);
""")

,fecha,periodo,origen,destino,distancia,actividad_origen,actividad_destino,residencia,renta,edad,sexo,viajes,viajes_km,estudio_destino_posible,estudio_origen_posible
0,20230701,07,0309908,03058,2-10,frecuente,casa,03,<10,NA,NA,5.958,25.842,False,False
1,20230701,13,0313303,0313302,2-10,frecuente,casa,03,<10,65-100,mujer,12.300,42.325,False,False
2,20230701,23,07003,07042,2-10,frecuente,casa,07,10-15,45-65,hombre,10.363,74.705,False,False
3,20230701,11,0801908,0801507,2-10,frecuente,casa,08,10-15,45-65,hombre,4.042,20.206,False,False
4,20230701,00,0801906,0801907,0.5-2,frecuente,casa,08,>15,0-25,hombre,22.165,21.008,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,20230701,10,1913002,1913003,0.5-2,frecuente,trabajo_estudio,19,10-15,NA,NA,8.060,7.622,False,False
173,20230701,16,3205403,3205403,0.5-2,frecuente,trabajo_estudio,32,>15,45-65,mujer,3.840,5.268,False,False
174,20230701,08,46251_AM,46217_AM,2-10,frecuente,trabajo_estudio,46,10-15,NA,NA,2.496,16.389,False,False
175,20230701,19,4625008,4625003,2-10,frecuente,trabajo_estudio,46,10-15,NA,NA,7.840,18.351,False,False


In [37]:
con.execute("""
    COPY (
        SELECT
            * EXCLUDE (estudio_destino_posible, estudio_origen_posible),
            CASE estudio_destino_posible WHEN 'si' THEN TRUE ELSE FALSE END AS estudio_destino_posible,
            CASE estudio_origen_posible  WHEN 'si' THEN TRUE ELSE FALSE END AS estudio_origen_posible
        FROM read_csv_auto(
            'csvs/20230701_Viajes_distritos.csv',
            types = {
                'estudio_destino_posible' : 'VARCHAR',
                'estudio_origen_posible'  : 'VARCHAR'
            }
        )
        USING SAMPLE 0.01 % (BERNOULLI)
    ) TO 'parquet/20230701_Viajes_distritos.parquet' (FORMAT PARQUET);
""")

In [42]:
SQL("""
    -- Para poder ver los distintos tipos de valores de la columna estudio_destino_posible
    SELECT estudio_destino_posible, COUNT(*) AS total_rows,
     FROM read_csv_auto(
        'csvs/20230701_Viajes_GAU.csv',
        types = {'estudio_destino_posible': 'VARCHAR'}
    )
    GROUP BY estudio_destino_posible
    USING SAMPLE 0.01 % (BERNOULLI);
""")

,estudio_destino_posible,total_rows
0,si,8
1,no,781


In [43]:
con.execute("""
    COPY (
        SELECT
            * EXCLUDE (estudio_destino_posible, estudio_origen_posible),
            CASE estudio_destino_posible WHEN 'si' THEN TRUE ELSE FALSE END AS estudio_destino_posible,
            CASE estudio_origen_posible  WHEN 'si' THEN TRUE ELSE FALSE END AS estudio_origen_posible
        FROM read_csv_auto(
            'csvs/20230701_Viajes_GAU.csv',
            types = {
                'estudio_destino_posible' : 'VARCHAR',
                'estudio_origen_posible'  : 'VARCHAR'
            }
        )
        USING SAMPLE 0.01 % (BERNOULLI)
    ) TO 'parquet/20230701_Viajes_GAU.parquet' (FORMAT PARQUET);
""")

In [45]:
SQL("""
SELECT
            * EXCLUDE (estudio_destino_posible, estudio_origen_posible),
            CASE estudio_destino_posible WHEN 'si' THEN TRUE ELSE FALSE END AS estudio_destino_posible,
            CASE estudio_origen_posible  WHEN 'si' THEN TRUE ELSE FALSE END AS estudio_origen_posible
        FROM read_csv_auto(
            'csvs/20230701_Viajes_municipios.csv',
            types = {
                'estudio_destino_posible' : 'VARCHAR',
                'estudio_origen_posible'  : 'VARCHAR'
            }
        )
        USING SAMPLE 0.01 % (BERNOULLI)
""")

,fecha,periodo,origen,destino,distancia,actividad_origen,actividad_destino,residencia,renta,edad,sexo,viajes,viajes_km,estudio_destino_posible,estudio_origen_posible
0,20230701,00,03048,03018,2-10,no_frecuente,casa,03,10-15,NA,NA,3.337,26.994,False,False
1,20230701,00,04044_AM,04006,10-50,frecuente,trabajo_estudio,04,<10,45-65,NA,9.846,118.369,False,False
2,20230701,00,05019,05019,2-10,casa,frecuente,05,10-15,0-25,hombre,26.954,69.231,False,False
3,20230701,00,06011,06149,10-50,no_frecuente,casa,06,<10,0-25,NA,4.929,68.861,False,False
4,20230701,00,06903_AM,06903_AM,2-10,frecuente,frecuente,06,<10,45-65,NA,2.480,13.986,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185,20230701,23,46094,46250,2-10,no_frecuente,casa,46,10-15,25-45,mujer,3.797,28.254,False,False
1186,20230701,23,46202,46147,2-10,frecuente,no_frecuente,46,10-15,45-65,hombre,2.630,9.759,False,False
1187,20230701,23,48015,48020,0.5-2,trabajo_estudio,casa,48,10-15,45-65,hombre,6.448,10.945,False,False
1188,20230701,23,48026_AM,48027,10-50,no_frecuente,casa,48,10-15,45-65,NA,2.564,39.799,False,False


In [44]:
con.execute("""
    COPY (
        SELECT
            * EXCLUDE (estudio_destino_posible, estudio_origen_posible),
            CASE estudio_destino_posible WHEN 'si' THEN TRUE ELSE FALSE END AS estudio_destino_posible,
            CASE estudio_origen_posible  WHEN 'si' THEN TRUE ELSE FALSE END AS estudio_origen_posible
        FROM read_csv_auto(
            'csvs/20230701_Viajes_municipios.csv',
            types = {
                'estudio_destino_posible' : 'VARCHAR',
                'estudio_origen_posible'  : 'VARCHAR'
            }
        )
        USING SAMPLE 0.01 % (BERNOULLI)
    ) TO 'parquet/20230701_Viajes_municipios.parquet' (FORMAT PARQUET);
""")

In [48]:
df = duckdb.read_parquet('parquet/20230701_Viajes_municipios.parquet')
df.fetchdf()

,fecha,periodo,origen,destino,distancia,actividad_origen,actividad_destino,residencia,renta,edad,sexo,viajes,viajes_km,estudio_destino_posible,estudio_origen_posible
0,20230701,00,02003,02003,0.5-2,frecuente,casa,02,>15,65-100,mujer,25.500,27.513,False,False
1,20230701,00,03015,03034_AM,2-10,trabajo_estudio,no_frecuente,30,10-15,0-25,mujer,5.796,31.411,False,False
2,20230701,00,03031,03031,0.5-2,no_frecuente,no_frecuente,46,10-15,0-25,hombre,11.678,12.305,False,False
3,20230701,00,03093,03093,0.5-2,frecuente,no_frecuente,03,10-15,NA,NA,2.192,2.505,False,False
4,20230701,00,04013,04013,2-10,frecuente,frecuente,04,<10,65-100,hombre,2.796,10.988,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209,20230701,23,48001_AM,48027,2-10,frecuente,casa,48,10-15,NA,NA,2.553,6.750,False,False
1210,20230701,23,48040_AM,48069,2-10,frecuente,frecuente,48,>15,NA,NA,5.022,20.272,False,False
1211,20230701,23,50153_AM,50160,10-50,trabajo_estudio,casa,50,10-15,NA,NA,3.900,49.709,False,False
1212,20230701,23,50297,50297,0.5-2,casa,trabajo_estudio,50,>15,0-25,mujer,10.390,13.586,False,False


In [ ]:
# 20230701_Personas_dia_distritos.csv
SQL("""
    SELECT
        * EXCLUDE (personas),
        CAST(REPLACE(personas, '.', '') AS INTEGER) AS personas
    FROM read_csv_auto(
        'csvs/20230701_Personas_dia_distritos.csv',
        delim = '|',
        columns = {
            'fecha': 'VARCHAR',
            'zona_pernoctacion': 'VARCHAR',
            'edad': 'VARCHAR',
            'sexo': 'VARCHAR',
            'numero_viajes': 'VARCHAR',
            'personas': 'VARCHAR'
        }
    ) AS cols;
""")

,fecha,zona_pernoctacion,edad,sexo,numero_viajes,personas
0,20230701,01001,0-25,hombre,0,158595
1,20230701,01001,0-25,hombre,1,11260
2,20230701,01001,0-25,hombre,2,80602
3,20230701,01001,0-25,hombre,2+,192506
4,20230701,01001,0-25,mujer,0,46601
...,...,...,...,...,...,...
119123,20230701,5200108,65-100,hombre,2+,783988
119124,20230701,5200108,65-100,mujer,0,538812
119125,20230701,5200108,65-100,mujer,1,125366
119126,20230701,5200108,65-100,mujer,2,333999
